# WHO Air quality database

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import re
import itertools as itools
from scipy import stats

## Prepare Data

In [ ]:
def Trim_Region(value : str)->str:
    r"""
    summary:
        remove LMIC, HIC, and Sub-Saharan
    """
    return re.sub('\s*(\(HIC\)|\(LMIC\)|\(Sub-Saharan\))', '', value)


def CalculateValues(orgvalue)->int:
    strvalue = str(orgvalue)
    match = re.match(r'\((\d+)\)-converted value', strvalue)
    if match:
        return int(match.group(1))
    return int(strvalue)

def GetIncome(Region : str)->str:
    return re.match('.+\((\w+)\)$', str(Region)).group(1)

def GetStationCount(value: str)->int:
    count = 0
    strvalue = str(value)
    for match in re.findall(r'(\d+)\s+\D+', strvalue):
        count += int(match)
    return count

def ChangeCountryName(country)->str:
    if country == 'Bolivia, Plurinational States of':
        return 'Bolivia'
    elif country == 'Iran (Islamic Republic of)':
        return 'Iran'
    elif country == 'The former Yugoslav Republic of Macedonia':
        return 'Macedonia'
    elif country == 'Russian Federation':
        return 'Russia'
    elif country == 'United Republic of Tanzania':
        return 'Tanzania'
    elif country == 'Venezuela, Bolivarian Republic of':
        return 'Venezuela'
    elif country == 'Viet Nam':
        return 'Vietnam'
    else:
        return country


data = pd.read_csv('database.csv', sep=';', skiprows=2)
data = data.drop(['status', 'Database version (year)', 'Reference for air quality', 'iso3'], axis = 1)

data['Income'] = [GetIncome(value) for value in data.Region]
data['Region'] = [Trim_Region(value) for value in data.Region]
data['Stationcount'] = [GetStationCount(value) for value in data['Number and type of monitoring stations']]
data['Country'] = [ChangeCountryName(value) for value in data.Country]
data = data.rename(columns={'City/Town' : 'City'})

pm10data= data.filter(['Region', 'Country', 'City', 'Year', 'Annual mean, ug/m3', 'Temporal coverage', 'note on converted PM10', 'Stationcount', 'Income'], axis=1)
pm10data['Converted'] = pm10data['note on converted PM10'].str.contains("Converted")
pm10data['Value'] = [CalculateValues(value) for value in pm10data['Annual mean, ug/m3']]
pm10data = pm10data.drop(['note on converted PM10', 'Annual mean, ug/m3'], axis = 1)
pm10data = pm10data.rename(columns={ 'Temporal coverage' : 'Coverage'})


pm25data= data.filter(['Region', 'Country', 'City', 'Year', 'Annual mean, ug/m3.1', 'Temporal coverage.1', 'note on converted PM2.5', 'Stationcount', 'Income'], axis=1)
pm25data= pm25data.rename(columns={ 'Temporal coverage.1' : 'Coverage'})
pm25data['Converted'] = pm25data['note on converted PM2.5'].str.contains("Converted")
pm25data['Value'] = [CalculateValues(value) for value in pm25data['Annual mean, ug/m3.1']]
pm25data = pm25data.drop(['note on converted PM2.5', 'Annual mean, ug/m3.1'], axis = 1)

pm10data.head()

## Get Country and City Information

In [ ]:

_countries = np.zeros(0)

def GetCountries()->list:
    global _countries
    if len(_countries) == 0:
        _countries = data.Country.unique()
    return _countries
_cities = np.zeros(0)
def GetCities()->list:
    global _cities
    if len(_cities) == 0:
        _cities = data.City.unique()
    return _cities
def GetCitiesOfCountry(country: str)->np.ndarray:
    return data.loc[data.Country == country].City.unique()

def GetDataForCountry(country: str)->tuple:
    return pm10data.loc[pm10data.Country == country], pm25data.loc[pm25data.Country == country]

def GetDataForCity(City: str, GetPM10 = True, GetPM25 = True)->tuple:
    if GetPM10 and GetPM25:
        return pm10data.loc[pm10data.City == City], pm25data.loc[pm25data.City == City]
    elif GetPM10:
        return pm10data.loc[pm10data.City == City]
    elif GetPM25:
        return pm25data.loc[pm25data.City == City]


GetCountries()

### Statistics for Country

In [ ]:
tablewidth = 80

def PrintHeader(country: str):
    headerstr = GetHeaderString(country)
    length = len(headerstr)
    PrintHashtagLine(length)
    print(headerstr)
    PrintHashtagLine(length)

def GetHeaderString(country: str)->str:
    return '# Information for %s #' % country

def PrintHashtagLine(length: str):
    print(''.join(itools.repeat('#', length)))

def PrintNumberOfCities(data: pd.DataFrame):
    print('Cities: %9d' % len(data.City.unique()))

def PrintYear(data: pd.DataFrame)->np.ndarray:
    years = data.Year.unique()
    print('No. of years: %3d' % len(years))
    print('First year:  %d' % min(years))
    print('Last year:   %d' % max(years))
    return np.sort(years)

def PrintEmptyLine():
    print(''.join(itools.repeat('-', linelength)))

def PrintTableLine():
    print('------+-------+-------+-----------------+-------+-----------------+-------------')

def PrintTableHeader(limit = 40):
    print('      | Count |           Min           |           Max           | Limit (%dug)' % limit)

def GetWhitespace(length: int)->str:
    return (('%%%ds' % length) % '')

def GetDashLine(length: int)->str:
    return ''.join(itools.repeat('-', length))

def PrintTableTitle(header: str):
    headerlength = len(header) + 14
    whitespacelength = (tablewidth - headerlength) / 2
    whitespace = GetWhitespace(whitespacelength)
    dashedline = GetDashLine(headerlength)
    print('%s%s' % (whitespace, dashedline))
    print('%s--     %s     --' % (whitespace, header))
    print('%s%s' % (whitespace, dashedline))


def PrintSingleStatic(data: pd.DataFrame, years: np.ndarray, header = 'PM10', limit = 40, measuredonly = True):
    if measuredonly:
        data = data.loc[data.Converted == False]
    
    if len(data)== 0:
        return

    PrintTableTitle(header)
    print()
    PrintTable(data, years, limit)

def PrintTable(data, years, limit):
    PrintTableHeader(limit)
    PrintYearStatistic(data, 'total', limit)
    for year in years:
        PrintYearStatistic(data.loc[data.Year == year], str(year), limit)
    PrintTableLine()


def PrintYearStatistic(data: pd.DataFrame, year : str, limit):
    if len(data) > 0:
        count = len(data)
        minvalue = min(data.Value)
        maxvalue = max(data.Value)
        mincities = data.loc[data.Value == minvalue].City.unique()
        maxcities = data.loc[data.Value == maxvalue].City.unique()
        limitcities = len(data.loc[data.Value > limit].City.unique())
        totalcities = len(data.City.unique())
        PrintTableLine()

        for yearval, countval, minval, mincity, maxval, maxcity, limit in itools.zip_longest(['%5s' % year], ['%5d' % count], ['%5d' % minvalue], mincities, ['%5d' % maxvalue], maxcities, ['%3d/%3d' % (limitcities, totalcities)], fillvalue='     '):
            if(len(mincity) > 15):
                mincity = mincity[0:15]
            if(len(maxcity) > 15):
                maxcity = maxcity[0:15]
            print('%s | %s | %s | %15s | %s | %15s | %10s' % (yearval, countval, minval, mincity, maxval, maxcity, limit))

    


def PrintStatistics(country: str, measuredonly = True):
    if country in GetCountries():
        _PrintStatistics(country, measuredonly)
    else:
        print('No data found for %s' % country)

def _PrintStatistics(country: str, measuredonly = True):
    PrintHeader(country)
    print()
    pm10, pm25 = GetDataForCountry(country)
    PrintNumberOfCities(pm10)
    years = PrintYear(pm10)
    PrintSingleStatic(pm10, years)
    print()
    PrintSingleStatic(pm25, years, header='PM2.5', limit = 25, measuredonly=measuredonly)

In [ ]:
PrintStatistics('Russia', False)

### Plot values and regression

In [ ]:
def _ShowRegression(xvalues, yvalues, color, header):
    slope, intercept, *rest = stats.stats.linregress(xvalues, yvalues)
    plt.plot(xvalues, yvalues, 'x', c=color)
    plt.plot(xvalues, intercept + xvalues * slope, c=color, label=header)

def PlotValue(data, color, header):
    xvalues = data.Year.values
    yvalues = data.Value.values
    _ShowRegression(xvalues, yvalues, color, header)

def PlotBoth(city: str):
    if city in GetCities():
        pm10, pm25 = GetDataForCity(city)
        if len(pm10) > 0:
            PlotValue(pm10, 'r', 'PM10')
        if len(pm25) > 0:
            PlotValue(pm25, 'b', 'PM2.5')
        plt.legend()
        plt.show()
    else:
        print('No data found for %s' % city)

PlotBoth('Shijiazhuang')

In [ ]:
def PlotCity(city: str, color, UsePM10):
    data = GetDataForCity(city, UsePM10, not UsePM10)
    if len(data) > 0:
        _ShowRegression(data.Year.values, data.Value.values, color, city)

def PlotMultiCities(cities:list, color:list, UsePM10 = True):
    for city, color in zip(cities, color):
        PlotCity(city, color, UsePM10)
    plt.legend()
    plt.show()
PlotMultiCities(['Bielefeld', 'Berlin', 'New York', 'Beijing'], ['r', 'b', 'g', 'c'], False)

### Rank by regression

In [ ]:
def GetSlopeInterceptCity(city: str, usepm10, measuredonly, mindatapoints)->tuple:
    data = GetDataForCity(city, usepm10, not usepm10)
    if measuredonly:
        data = data.loc[data.Converted == False]
    if len(data) < mindatapoints:
        return False, 0, 0, city, ''
    else:
        slope, intercept, *rest = stats.stats.linregress(data.Year.values, data.Value.values)
        return True, slope, intercept, city, data.Country.values[0]

def GetSlopeInterceptAll(usepm10, measuredonly, mindatapoints)->list:
    return [(slope, intercept, city, country) for s, slope, intercept, city, country in _GetSlopeInterceptAll(usepm10, measuredonly, mindatapoints) if s]
def _GetSlopeInterceptAll(usepm10, measuredonly, mindatapoints)->list:
    return [GetSlopeInterceptCity(city, usepm10, measuredonly, mindatapoints) for city in GetCities()]

def PrintSlopeRanking(UsePM10, MeasuredOnly, Reverse, count = 15):
    print()
    data = GetSlopeInterceptAll(UsePM10, MeasuredOnly, 3)
    data.sort(key=lambda tup: tup[0], reverse=Reverse)
    data = data[:count]
    length = max([len('%s (%s)'  % (city, country)) for slope, intercept, city, country in data])
    valuelength = max([len('%.2f' % slope) for slope, intercept, city, country in data])
    numberlength = len(str(count))
    format = '%%%dd. %%%ds %%%ds' % (numberlength, length, valuelength)
    for i, cdata in enumerate(data):
        strvalue = '%.2f' % (cdata[0])
        print(format % (i+1, '%s (%s)'  % (cdata[2], cdata[3]), strvalue))

PrintSlopeRanking(False, False, False)